## Starter Notebook
EDA Notebook on Cryptcurrency Data
There are 26000+ csv files in the current version of the dataset:

In [335]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np
import os # for accessing directory structure
import pandas as pd
import glob

In [336]:
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#         print(os.path.join(dirname, filename))


# Above code will print it all like below, this was just for the initial checking
# I am commenting out as this folder has 26000+ file names to pring
# /kaggle/input/crypto-data/cr_20170822-152505.csv
# /kaggle/input/crypto-data/cr_20170812-020505.csv
# /kaggle/input/crypto-data/cr_20170813-065506.csv
# /kaggle/input/crypto-data/cr_20171005-012506.csv

In [337]:
# Distribution graphs (histogram/bar graph) of column data
def plot_per_column_distribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()


In [338]:
# Correlation matrix
def plot_correlation_matrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [339]:
# Scatter and density plots
def plot_scatter_matrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [340]:
# Define varable for the first 3 files

nRowsRead = 1000 # specify 'None' if want to read whole file
# These .csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('./kaggle/input/crypto-data/cr_20170804-034052.csv', delimiter=',', nrows = nRowsRead)
df2 = pd.read_csv('./kaggle/input/crypto-data/cr_20170804-035004.csv', delimiter=',', nrows = nRowsRead)
df3 = pd.read_csv('./kaggle/input/crypto-data/cr_20170804-040006.csv', delimiter=',', nrows = nRowsRead)

In [341]:
# Let's check 1st file: /kaggle/input/crypto-data/cr_20170804-034052.csv
nRowsRead = 1000 # specify 'None' if want to read whole file
# cr_20170804-034052.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
# df1.dataframeName = 'cr_20170804-034052.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 10 columns


In [342]:
# Let's take a quick look at what the data looks like:
df1.head(5)

,symbol,ranking by market cap,name,market cap,price,circulating supply,volume,% 1h,% 24h,% 1wk
0,GMB,1011,Gambleo,?,$0.000895,?,*,Low Vol,?,?
1,FTC,164,Feathercoin,"$9,481,236",$0.055794,"169,932,600","$1,267,140",-0.05%,10.49%,4.99%
2,HCC,953,Happy Creator...,?,$0.000108,?,*,Low Vol,-0.02%,8.39%
3,BRX,288,Breakout Stake,"$1,793,819",$0.286183,"6,268,082",*,"$4,778",-4.53%,3.35%
4,UNRC,872,UniversalRoya...,?,$0.005857,?,*,$664,3.98%,-7.83%


Distribution graphs (histogram/bar graph) of sampled columns:

In [343]:
plot_per_column_distribution(df1, 10, 5)

<Figure size 2400x512 with 0 Axes>

In [344]:
# Let's check 2nd file: /kaggle/input/crypto-data/cr_20170804-035004.csv
df2.dataframeName = 'cr_20170804-035004.csv'
nRow, nCol = df2.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 10 columns


In [345]:
# Let's take a quick look at what the data looks like:
df2.head(5)

,symbol,ranking by market cap,name,market cap,price,circulating supply,volume,% 1h,% 24h,% 1wk
0,XBY,190,XtraBYtes,"$7,422,935",$0.011420,"650,000,000",*,"$16,273",-1.36%,-0.03%
1,CPN,637,CompuCoin,"$57,962",$0.002955,"19,615,019",Low Vol,0.34%,-4.69%,-31.49%
2,CHEAP,976,Cheapcoin,?,$0.000112,?,*,Low Vol,0.02%,105.40%
3,BLAS,700,BlakeStar,"$27,044",$0.000112,"242,418,240",Low Vol,0.02%,-5.80%,?
4,CPC,341,Capricoin,"$848,890",$0.426568,"1,990,045",*,"$18,457",-0.49%,3.66%


Distribution graphs (histogram/bar graph) of sampled columns:

In [346]:
plot_per_column_distribution(df2, 10, 5)

<Figure size 2400x512 with 0 Axes>

In [347]:
# Let's check 3rd file: /kaggle/input/crypto-data/cr_20170804-040006.csv
df3.dataframeName = 'cr_20170804-040006.csv'
nRow, nCol = df3.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 10 columns


In [348]:
# Let's take a quick look at what the data looks like:
df3.head(5)

,symbol,ranking by market cap,name,market cap,price,circulating supply,volume,% 1h,% 24h,% 1wk
0,XBY,192,XtraBYtes,"$7,322,055",$0.011265,"650,000,000",*,"$16,282",-1.93%,-1.53%
1,CPN,638,CompuCoin,"$57,975",$0.002956,"19,615,019",Low Vol,0.36%,-4.67%,-31.38%
2,CHEAP,978,Cheapcoin,?,$0.000112,?,*,Low Vol,0.17%,105.40%
3,BLAS,701,BlakeStar,"$27,038",$0.000112,"242,418,240",Low Vol,0.05%,-7.01%,?
4,CPC,341,Capricoin,"$849,276",$0.426762,"1,990,045",*,"$18,466",-0.51%,3.91%


In [349]:
# Distribution graphs (histogram/bar graph) of sampled columns:
plot_per_column_distribution(df3, 10, 5)

<Figure size 2400x512 with 0 Axes>

In [350]:
print(df1.shape)
print(df1.dtypes)

(1000, 10)
symbol                   object
ranking by market cap     int64
name                     object
market cap               object
price                    object
circulating supply       object
volume                   object
% 1h                     object
% 24h                    object
% 1wk                    object
dtype: object


In [351]:
# lets print all the files in the directory.
file_dir = './kaggle/input/crypto-data'
!ls $file_dir | wc -l

24


In [352]:
# Define a variable for to hold a Python-list of .csv files in that directory
# files_list = glob.glob(os.path.join(file_dir, "*.csv"))
all_files = glob.glob(os.path.join(file_dir, "*.csv"))

# lets take the first 20 .csv file (from which I shall create a combined-dataframe)
files_list = all_files[:20]

# lets create dataframes and print them to see if it working

df1 = pd.read_csv(files_list[0])
df2 = pd.read_csv(files_list[1])
df3 = pd.read_csv(files_list[2])

print(df1.head(), "\n")
print(df2.head(), "\n")
print(df3.head(), "\n")

  symbol  ranking by market cap       name   market cap      price  \
0    XBY                    185  XtraBYtes  $7,819,695   $0.012030   
1    CPN                    642  CompuCoin     $57,921   $0.002953   
2  CHEAP                    977  Cheapcoin           ?   $0.000111   
3   BLAS                    699  BlakeStar     $27,016   $0.000111   
4    CPC                    344  Capricoin    $854,157   $0.429215   

  circulating supply   volume     % 1h    % 24h    % 1wk  
0        650,000,000        *  $18,438    7.06%    5.37%  
1         19,615,019  Low Vol    0.18%   -7.21%  -27.36%  
2                  ?        *  Low Vol    0.18%  105.12%  
3        242,428,320  Low Vol    0.36%  -17.94%        ?  
4          1,990,045        *  $15,219   -0.01%    2.74%   

  symbol  ranking by market cap       name   market cap      price  \
0    XBY                    193  XtraBYtes  $7,328,230   $0.011274   
1    CPN                    640  CompuCoin     $57,999   $0.002957   
2  CHEAP     

All these files have the same columns so it seems reasonable to concatenate everything into one dataframe.

- First, creating a list of dataframes with the filenames in a "file_name" column
- Then concatenate them all into one big dataframe

In [353]:
dataframes = [pd.read_csv(file).assign(file_name=os.path.basename(file).strip(".csv")) for file in files_list]
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.head()

,symbol,ranking by market cap,name,market cap,price,circulating supply,volume,% 1h,% 24h,% 1wk,file_name
0,XBY,185,XtraBYtes,"$7,819,695",$0.012030,"650,000,000",*,"$18,438",7.06%,5.37%,r_20170804-070005
1,CPN,642,CompuCoin,"$57,921",$0.002953,"19,615,019",Low Vol,0.18%,-7.21%,-27.36%,r_20170804-070005
2,CHEAP,977,Cheapcoin,?,$0.000111,?,*,Low Vol,0.18%,105.12%,r_20170804-070005
3,BLAS,699,BlakeStar,"$27,016",$0.000111,"242,428,320",Low Vol,0.36%,-17.94%,?,r_20170804-070005
4,CPC,344,Capricoin,"$854,157",$0.429215,"1,990,045",*,"$15,219",-0.01%,2.74%,r_20170804-070005


In [354]:
combined_df.shape

(20280, 11)

In [355]:
# Creating a dataframe, by filtering only the data where the row is BTC
btc_df = combined_df[combined_df['symbol'] == 'BTC']
btc_df.shape

(0, 11)

In [356]:
btc_df.to_csv("combined-btc.csv")
